In [25]:
import numpy as np
from sklearn import datasets
from sklearn import tree
from sklearn import model_selection
from sklearn import ensemble
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import KFold, cross_validate
from sklearn.base import clone

We work with the California housing prices dataset.

In [4]:
california_housing = datasets.fetch_california_housing()
print(california_housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

Split the dataset for validation set method.

In [5]:
y = california_housing.target
X = california_housing.data
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.9, random_state=10)

First, fit a single regression tree without any size limits or pruning.

In [6]:

model1 = tree.DecisionTreeRegressor(random_state=1)
model1.fit(X_train, y_train)

,"criterion criterion: {""squared_error"", ""friedman_mse"", ""absolute_error"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in the half mean Poisson deviance to find splits... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 0.24 Poisson deviance criterion.",'squared_error'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.For an example of how ``max_depth`` influences the model, see:ref:`sphx_glr_auto_examples_tree_plot_tree_regression.py`.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",1
,"max_le

In [7]:
print(model1.score(X_train, y_train))
print(model1.score(X_test, y_test))

1.0
0.5919969805777842


The score is the $R^2$ of the model. Since we didn't control the tree-size or prune the tree, for the training data $R^2 = 1$. For the tree to generalize better, we try both those methods: 

(1) Control the minimum data points at each node prior to the split:

In [8]:
model2 = tree.DecisionTreeRegressor(min_samples_split=30, random_state=1)
model2.fit(X_train, y_train)

,"criterion criterion: {""squared_error"", ""friedman_mse"", ""absolute_error"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in the half mean Poisson deviance to find splits... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 0.24 Poisson deviance criterion.",'squared_error'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.For an example of how ``max_depth`` influences the model, see:ref:`sphx_glr_auto_examples_tree_plot_tree_regression.py`.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",30
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",1
,"max_l

In [9]:
print(model2.score(X_train, y_train))
print(model2.score(X_test, y_test))

0.8724056925744369
0.7171389906807539


Constraining each node to have atleast $15$ data points gives a much better test performance.

(2) Cost complexity pruning:

In [17]:
model = tree.DecisionTreeRegressor(random_state=1)
ccp_alphas = model.cost_complexity_pruning_path(X_train, y_train).ccp_alphas

min_ccp_alpha, max_ccp_alpha = min(ccp_alphas), max(ccp_alphas)
ccp_alphas = np.arange(min_ccp_alpha, max_ccp_alpha, (max_ccp_alpha - min_ccp_alpha) / 100)
model3, model3_score = None, 0

for ccp_alpha in ccp_alphas:
    model = tree.DecisionTreeRegressor(ccp_alpha=ccp_alpha, random_state=1)
    model.fit(X_train, y_train)
    _score = model.score(X_test, y_test)
    if _score > model3_score:
        model3 = model
        model3_score = _score

print(model3_score)

0.6246809575876497


Next, we try bagging (bootstrap aggregating). Instead of a single regression tree, we train multiple trees on bootstrap samples of the dataset. The average of the predicted value across all the trees is the final average.

In [11]:
model_bagging = ensemble.BaggingRegressor(estimator=tree.DecisionTreeRegressor(random_state=1), n_estimators=500)
model_bagging.fit(X_train, y_train)
print(model_bagging.score(X_test, y_test))

0.8219637907916783


Note that we grow out each tree to the fullest and then take the average output. This gives a much higher $R^2$ as expected.

Next, try the Random Forest regressor which is an improvement over bagging in that every tree uses a random subset of the predictors. In essence, this reduces the correlation between the trees.

In [12]:
model_forest = ensemble.RandomForestRegressor(n_estimators=700)
model_forest.fit(X_train, y_train)
model_forest.score(X_test, y_test)

0.8216840343822075

In [ ]:
def fit_and_score(estimator, X_train, X_test, y_train, y_test):
    """Fit the estimator on the train set and score it on both sets"""
    estimator.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False);

    train_score = estimator.score(X_train, y_train)
    test_score = estimator.score(X_test, y_test)

    return estimator, train_score, test_score


cv = KFold(n_splits=10, shuffle=True, random_state=1)

reg = xgb.XGBRegressor(tree_method="hist")

results = {}
best_test_score = float('inf')
best_est = None

for train, test in cv.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]
    est, train_score, test_score = fit_and_score(
        clone(reg), X_train, X_test, y_train, y_test
    )
    results[est] = (train_score, test_score)
    if test_score < best_test_score:
        best_est = est

results[best_est]

[0]	validation_0-rmse:0.94551
[1]	validation_0-rmse:0.80656
[2]	validation_0-rmse:0.71749
[3]	validation_0-rmse:0.66402
[4]	validation_0-rmse:0.62330
[5]	validation_0-rmse:0.60408
[6]	validation_0-rmse:0.58496
[7]	validation_0-rmse:0.57339
[8]	validation_0-rmse:0.56327
[9]	validation_0-rmse:0.55271
[10]	validation_0-rmse:0.54165
[11]	validation_0-rmse:0.53716
[12]	validation_0-rmse:0.53074
[13]	validation_0-rmse:0.52472
[14]	validation_0-rmse:0.51770
[15]	validation_0-rmse:0.51706
[16]	validation_0-rmse:0.51503
[17]	validation_0-rmse:0.51238
[18]	validation_0-rmse:0.50914
[19]	validation_0-rmse:0.50856
[20]	validation_0-rmse:0.50650
[21]	validation_0-rmse:0.50556
[22]	validation_0-rmse:0.50520
[23]	validation_0-rmse:0.50484
[24]	validation_0-rmse:0.50307
[25]	validation_0-rmse:0.49851
[26]	validation_0-rmse:0.49493
[27]	validation_0-rmse:0.49426
[28]	validation_0-rmse:0.49302
[29]	validation_0-rmse:0.49032
[30]	validation_0-rmse:0.48976
[31]	validation_0-rmse:0.48896
[32]	validation_0-

(0.9406431972289525, 0.8406936631429096)